<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to COMP 680 #
### Statistics for Computing and Data Science ###

### Week 12 Generalized Linear Models ###

# Generalized Linear Models

In this demo, we will be using Python libaray [statsmodels](https://www.statsmodels.org/stable/index.html) for Generalized Linear Models [**GLM**](https://www.statsmodels.org/stable/glm.html).

In [ ]:
### standard imports
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.sandbox.predict_functional import predict_functional

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
plt.rcParams["figure.figsize"] = (8,6)
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## Logistic Regression

The logistic regression model predicts the **probability** that the **binary** outcome variable $Y$. It is probably one of the most widely used basic models for classification and is a simple extension of linear models to the classification problem. The basic idea is to model the log odds ratio of $Y$ using a linear function of all the features $x$.

$$
\log \frac{\mathbb{P}\left(Y=1 \,|\, x\right)}{\mathbb{P}\left(Y=0 \,|\, x\right)} = \beta_0 + \beta_1 x_1 + \cdots + \beta_p x_p
$$

Just as with our linear regression model, it is **parameterized** by the intercept $\beta_0$ and slopes $\beta_1, \cdots \beta_p$ for all the features. 

We can rewrite the logistic regression model in the form:
$$
\mathbb{P} \left(Y=1 \,|\, x\right) =\text{Logit}\left(\beta_0 + \beta_1 x_1 + \cdots \beta_p x_p\right)
$$

where the logit function maps the output of the linear model (which could be any real number) to the interval between 0 and 1  and is commonly used when modeling probabilities. 

### Obtaining the Data

For this part, we will use the [Wisconsin Breast Cancer Dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)) which we can obtain from [`scikit learn`](https://scikit-learn.org/stable/index.html). This dataset consists of measurements from tumor biopsies for 569 patients as well as whether the tumor was malignant or benign. It includes 212 malignant cases and 357 benign cases.

In [ ]:
import sklearn.datasets
# load the dataset
breast_cancer = sklearn.datasets.load_breast_cancer()
# create a dataframe
data = pd.DataFrame(breast_cancer['data'], columns=breast_cancer['feature_names'])
print(data.shape)
data.head()

### The Outcome Variable

The goal is to predict whether a tumor is benign or malignant (a binary decision) given characteristics of that tumor. In `sklearn` the outcome variable or data label is captured by the column named `"target"`.  To put the data back in it's original context we will create a new column called `"malignant"` which will be 1 if the tumor is malignant and 0 if it is benign.

In [ ]:
# Target breast_cancer['target'] = 0 is malignant 1 is benign
# Create a new Y because the model predicts the probability of Y=1
data['malignant'] = (breast_cancer['target']==0)*1
data['malignant'].value_counts()

### Selecting Features

What features might be a good indication of whether a tumor is benign or malignant? 

In [ ]:
# print out all the column names so I can copy and paste in model formula
print(data.columns)
# select a list of columns that are relevant
col_list = ['mean radius', 'mean texture', 'mean perimeter', 'mean area','mean smoothness', 
             'mean compactness', 'mean concavity','mean concave points', 'mean symmetry', 
             'mean fractal dimension','malignant']
dat = data[col_list]
# rename the columns to get rid of the space...there should be work around this
# but the statsmodel formula does not like any space in variable names
dat.rename(columns={'mean radius':'mean_radius', 'mean texture':'mean_texture', 
                     'mean perimeter':'mean_perimeter', 'mean area':'mean_area',
                     'mean smoothness':'mean_smoothness', 'mean compactness':'mean_compactness', 
                     'mean concavity':'mean_concavity', 'mean concave points':'mean_concave_points', 
                     'mean symmetry':'mean_symmetry', 'mean fractal dimension':'mean_fractal_dimension'}, inplace=True)
dat.head()

Just to explore our data, let's create scatter plot matrix, but color code based on outcome variable. This can be easily done by `seaborn.pariplot` using argument `hue`.

In [ ]:
# seaborn has very nice default setting and easy syntax
sns.pairplot(dat, diag_kind='hist', hue='malignant');

Perhaps a good starting point is the size of the tumor.  Larger tumors are probably more likely to be malignant.  In the following, we plot whether the tumor was malignant (1 or 0) against the `"mean radius"`, i.e., zoom in the first histogram:

In [ ]:
# overlaid histogram
tumor_bin = np.arange(5, 31, 0.5)
dat.loc[dat['malignant']==0, 'mean_radius'].plot(kind='hist',bins=tumor_bin,alpha=0.4,label='benign')
dat.loc[dat['malignant']==1, 'mean_radius'].plot(kind='hist',bins=tumor_bin,alpha=0.4,label='malignant')
plt.xlabel('tumor mean radius')
plt.legend();

In [ ]:
# side by side box plot
sns.boxplot(data=dat, x='malignant', y='mean_radius')
plt.title('tumor radius vs. malignant status');

### Simple Logistic Regression Model

To get an intuition for the behavior of the parameters in the logistic regression model let's consider a simple one dimensional logistic regression model of the form:

$$
\mathbb{P}\left(Y=1 \,|\, x\right) =\sigma\left(\beta_0 + \beta_1 x\right) = \frac{1}{1+\exp\left(-\beta_0 - \beta_1 x\right)}
$$

In [ ]:
model1 = sm.GLM.from_formula('malignant ~ mean_radius', 
                             family=sm.families.Binomial(), data=dat)
result1 = model1.fit()
result1.summary()

How do we interpret the coefficients?
- intercept:
- slope:

In [ ]:
# print out parameters
result1.params

In [ ]:
# convert to odds ratio scale
np.exp(result1.params)

In [ ]:
# this is all fitted values, i.e. the probability of Y=1
result1.fittedvalues

In [ ]:
# this is the same as above, why?
result1.predict(dat['mean_radius'])

In [ ]:
# create a small dataframe to compare true Y vs. model predicted Y
df = dat[['malignant']]
df['prob1'] = result1.fittedvalues
df['pred1'] = (result1.fittedvalues>0.5)*1
# create 2 x 2 matrix
df.pivot_table(index='malignant', columns='pred1', values='prob1', aggfunc='count')

This pivot table is called [**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix) in machine learning. It is more common in multiclass classification problems. 
1. **Correct predictions** are on the diagonal.
1. **Incorrect predictions** are off the diagonal.

Some common evaluation metrics for binary classification:
- Accuracy rate: proportion of correct prediction.
- True positive rate (sensitivity, recall): proportion of correctly predicted 1 out of true 1.
- True negative rate (specificity): proportion of correctly predicted 0 out of true 0.
- Precision: proportion of correctly predicted 1 out of predicted 1.
- F1 score: harmonic mean of precision and recall.

### Add one more variable

In [ ]:
model2 = sm.GLM.from_formula('malignant ~ mean_radius + mean_texture', 
                             family=sm.families.Binomial(), data=dat)
result2 = model2.fit()
result2.summary()

In [ ]:
# how much does the model improve?
df['prob2'] = result2.fittedvalues
df['pred2'] = (result2.fittedvalues>0.5)*1
df.pivot_table(index='malignant', columns='pred2', values='prob1', aggfunc='count')

Let's plot the decision boundary with original data. What is the decision boundary?
$$ \mathbb{P}(\hat{Y}=1) = 0.5 \Rightarrow \hat{\beta_0} + \hat{\beta_1} X_1 + \hat{\beta_2}X_2 = 0 \Rightarrow  X_2 = - \frac{\hat{\beta_0}}{\hat{\beta_2}} - \frac{\hat{\beta_1}}{\hat{\beta_2}}X_1$$

In [ ]:
# print out parameters
result2.params

In [ ]:
# extract intercept and slopes
beta0_hat = result2.params['Intercept']
beta1_hat = result2.params['mean_radius']
beta2_hat = result2.params['mean_texture']

In [ ]:
# define a function to calculate the logistic decision boundary for rank 1
def logistic_decision(x, glm_result):
    beta0_hat = glm_result.params[0]
    beta1_hat = glm_result.params[1]
    beta2_hat = glm_result.params[2]
    y = -beta0_hat/beta2_hat-beta1_hat*x/beta2_hat
    return y

In [ ]:
# to plot X_1 (radius) on x_axis and X_2 (texture) on y_axis
x_min, x_max = dat['mean_radius'].min()-1, dat['mean_radius'].max()+1
y_min, y_max = dat['mean_texture'].min()-1, dat['mean_texture'].max()+1

# expand a grid on X and Y that contains my original data
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
X_grid = np.c_[xx.ravel(), yy.ravel()]

# tried glm_model.predict(X_grid) but did not work
pred_X_grid = (X_grid[:,1]- logistic_decision(X_grid[:,0], result2)) > 0
# plot the grid with predicted label
sns.scatterplot(x=X_grid[:,0], y=X_grid[:,1], hue=pred_X_grid, alpha=0.2, legend=False)


# create the the straight line for the decision boundary
X = np.arange(x_min, x_max, 0.1)
Y = logistic_decision(X, result2) #-beta0_hat/beta2_hat-beta1_hat*X/beta2_hat 
# plot the original data, color code with label
sns.scatterplot(x=dat['mean_radius'], y=dat['mean_texture'], hue=dat['malignant'])
# plot the decision boundary
sns.lineplot(x=X, y=Y, color='red', lw=4, label='decision boundary')
# restrict the Y lim makes the plot look nicer
plt.xlim((x_min, x_max))
plt.ylim((y_min, y_max))
plt.legend()
plt.title('Logistic Regression Decision Boundary');

### Add even more variables

In [ ]:
# screen some variables based on correlation
dat.corr()

In [ ]:
model3 = sm.GLM.from_formula('malignant ~ mean_radius+mean_texture+mean_smoothness+mean_compactness+mean_symmetry+mean_fractal_dimension', 
                             family=sm.families.Binomial(), data=dat)
result3 = model3.fit()
result3.summary()

In [ ]:
# how much does the model improve?
df['prob3'] = result3.fittedvalues
df['pred3'] = (result3.fittedvalues>0.5)*1
df.pivot_table(index='malignant', columns='pred3', values='prob1', aggfunc='count')

### Visualization of the fitted models

Visualization of fitted logistic regression models is more challenging than visualization of fitted linear models, but is still worth pursuing. 


We can begin by plotting the fitted proportion of the
population that smokes, for various subpopulations defined by the
regression model.  We will focus here on how the smoking rate varies
with age, so we restrict the population to female college graduates.

The following plot shows the fitted log odds (or logit) probability for breast cancer as a function of tumor radius.  The grey band is a 95% confidence band, similar as in the case of a linear model.

In [ ]:
tumor_mean = dat.mean()
tumor_mean

In [ ]:
from statsmodels.sandbox.predict_functional import predict_functional

# Fix certain variables at reference values
# Here we use average value in the dataset as reference
# but you can specify particular values

values = {'mean_texture':tumor_mean[1], 'mean_smoothness':tumor_mean[4], 
          'mean_compactness':tumor_mean[5], 'mean_symmetry':tumor_mean[8],
          'mean_fractal_dimension':tumor_mean[9]}
pred, CI, X = predict_functional(result3, 'mean_radius', values=values)

ax = sns.lineplot(X, pred, lw=5)
ax.fill_between(X, CI[:, 0], CI[:, 1], color='grey', alpha=0.4)
ax.set_xlabel('tumor radius')
ax.set_ylabel('log odds of cancer');

We can display the same plot interms of actual probability instead of log odds. A transformation by:

$$
\mathbb{P}(Y=1) = \frac{1}{1 + e^{- \text{log odds}}}
$$

In [ ]:
# original pred and CI are log odds
pred_prob = 1 / (1 + np.exp(-pred))
CI_prob = 1 / (1 + np.exp(-CI))
ax = sns.lineplot(X, pred_prob, lw=4)
ax.fill_between(X, CI_prob[:, 0], CI_prob[:, 1], color='grey', alpha=0.4)
ax.set_xlabel('tumor radius')
ax.set_ylabel('probability of cancer');

In [ ]:
# visualize a confusion matrix using sklearn
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df['malignant'], df['pred3'])
sns.heatmap(cm, annot=True, fmt='d', xticklabels=["benign","malignant"], 
            yticklabels=["benign", "malignant"]);

Evaluate a binary classifier:  
[Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall)  
[F1 score](https://en.wikipedia.org/wiki/F1_score)
[ROC curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(df['malignant'], df['prob3'])
fpr, tpr, thresholds = roc_curve(df['malignant'], df['prob3'])
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
# the 45 degree reference line
plt.plot([0, 1], [0, 1],'r--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

## Multinomial Regression
This is a straightforward extension on logistic regression, and in some software you will see them implemented under the same function as Logistic regression or GLM, simply using a family = Multinomial. 

In `statsmodels` can be implemented by [MNLogit](https://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.MNLogit.html):
- this does not support from_formula so we need to manually put in the dependent and independent variables
- it does not treat categorical variables properly so we need to manually create dummy variables

### Obtaining the Data

For this part, we will use a subset of data from the [World Values Surveys](https://www.worldvaluessurvey.org/wvs.jsp) 1995-1997 for Australia, Norway, Sweden, and the United States. This dataset has 5381 observations on the following 6 variables:
- poverty: 
    - “Do you think that what the government is doing for people in poverty in this country is about the right amount, too much, or too little?” (ordered): Too Little, About Right, Too Much
- religion: 
    - Member of a religion: no or yes.
- degree: 
    - Held a university degree: no or yes.
- country:
    - Australia
    - Norway
    - Sweden
    - USA
- age: in years
- gender: male or female

Source: J. Fox and R. Andersen (2006) Effect displays for multinomial and proportional-odds logit models. Sociological Methodology 36, 225–255.

In [ ]:
WVS = pd.read_csv('world_values_surveys.csv')
WVS.head()

In [ ]:
WVS.dtypes

First we treat the survey response to the question “Do you think that what the government is doing for people in poverty in this country is about the right amount, too much, or too little?” as nominal categorical variable, so ignore the ordering, and fit a multinomial regression model. 

In [ ]:
# need to manually expand all the dummy variable columns
# in this case, the baseline is non-religious, no university degree, American men
WVS['religious'] = (WVS['religion']=='yes')*1
WVS['university_degree'] = (WVS['degree']=='yes')*1
WVS['Australia'] = (WVS['country']=='Australia')*1
WVS['Norway'] = (WVS['country']=='Norway')*1
WVS['Sweden'] = (WVS['country']=='Sweden')*1
WVS['female'] = (WVS['gender']=='female')*1
# check dtypes again
WVS.dtypes

In [ ]:
# endog is the response variable
WVS.endog = WVS['poverty']
# exog is the design matrix: add a constant column of 1 for intercept
WVS.exog = sm.add_constant(WVS[['religious', 'university_degree' ,'age', 'female', 'Australia', 'Norway', 'Sweden']])
# model set up
model4 = sm.MNLogit(WVS.endog, WVS.exog)
# model fit
result4 = model4.fit()
# print result summary
result4.summary()

In [ ]:
# this model has no fittedvalues, so make prediction on X
# make prediction: output are probabilities
result4.predict(WVS.exog)
# how do we convert to categories: find the column index with max prob
result4.predict(WVS.exog).idxmax(axis=1)
# find which corresponding number of the categories
poverty_dict = {0:'About Right', 1:'Too Little', 2:'Too Much'}
WVS['poverty_pred'] = result4.predict(WVS.exog).idxmax(axis=1).replace(poverty_dict)
WVS.pivot_table(index='poverty', columns='poverty_pred', values='age', aggfunc='count')

Next we treat the survey response as an ordinal categorical variable, and fit a ordinal regression model. 

This can be implemented by [OrderedModel](https://www.statsmodels.org/dev/examples/notebooks/generated/ordinal_regression.html):
- [example](https://www.statsmodels.org/dev/examples/notebooks/generated/ordinal_regression.html)
- [source code](https://www.statsmodels.org/devel/_modules/statsmodels/miscmodels/ordinal_model.html)

Alternatively, I found this:
- [Mord package](https://pythonhosted.org/mord/)

In [ ]:
# If you get an error message: No module named "statsmodels.miscmodels.ordinal_model"
# then uncomment the pip install to upgrade your statsmodels package

#%pip install --upgrade --no-deps statsmodels 
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [ ]:
# convert the response to ordinal variable
WVS['poverty_ord'] = pd.Categorical(WVS['poverty'], categories=['Too Little', 'About Right', 'Too Much'], ordered=True)
WVS.head()

In [ ]:
# if the OrderedModel works as the example the following code should work
WVS.endog_ord = WVS['poverty_ord']
# ordinal regression model does not need constant for intercept, why?
WVS.exog_ord = WVS[['religious', 'university_degree' ,'age', 'female', 'Australia', 'Norway', 'Sweden']]

model5 = OrderedModel(WVS.endog_ord, WVS.exog_ord, distr='logit')
# alternatively, you can use distr='probit', so use probit link function
# probit link is also commonly used but does not provide interpretation as log odds ratio
result5 = model5.fit(method='bfgs', disp=False)
result5.summary()

## Poisson Regression

### Obtaining the Data 

In this part, we will use a dataset about species diversity on the Galapagos Islands. There are 30 Galapagos islands and 7 variables in the dataset. The relationship between the number of plant species and several geographic variables is of interest. The original dataset contained several missing values which have been filled for convenience.

- Species: the number of plant species found on the island
- Endemics: the number of endemic species, those that are restricted to a geographical area and do not occur naturally in any other part of the world.
- Area: the area of the island (km$^2$)
- Elevation: the highest elevation of the island (m)
- Nearest: the distance from the nearest island (km)
- Scruz: the distance from Santa Cruz island (km)
- Adjacent: the area of the adjacent island (square km)

Source: M. P. Johnson and P. H. Raven (1973) "Species number and endemism: The Galapagos Archipelago revisited" Science, 179, 893-895

In [ ]:
gala = pd.read_csv('gala.csv')
gala

In [ ]:
# endog is the response variable
gala.endog = gala['Species']
# exog is the design matrix: add a constant column of 1 for intercept
gala.exog = sm.add_constant(gala[['Endemics', 'Area', 'Elevation', 'Nearest', 'Scruz','Adjacent']])
# model set up
model6 = sm.Poisson(gala.endog, gala.exog)
# model fit
result6 = model6.fit(method='newton')
# print out result
result6.summary()

For your curiosity, the same data but fitting Negative Binomial regression instead of Poisson. This is more flexible due to the additional dispersion parameter. 

In [ ]:
model7 = sm.NegativeBinomial(gala.endog, gala.exog)
result7 = model7.fit(method='newton')
result7.summary()